In [ ]:
# default_exp projects.self_supervised

# Self-supervised Learning
> Self-supervised utility code

In [ ]:
# hide
from nbdev.showdoc import *
from fastcore.test import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# export
# import pandas as pd
# import numpy as np

from fastai.vision.all import *
from self_supervised.layers import *
from self_supervised.models.vision_transformer import *
from scp.utils.general import custom_load

## Model splitter functions

Split the model architecture at certain points to allow for discriminative learning rates. 

In [ ]:
# export
def model_split1(model):
    '''Basic feature extractor & FC head split for '''
    groups = L([model.vit_backbone, model.mlp])
    return groups.map(params)

def model_split2(model):
    '''Attempt to do a split like in fastai i.e. feature extractor is split in the middle'''
    g1 = nn.Sequential(model.vit_backbone.patch_embed, model.vit_backbone.pos_drop, model.vit_backbone.blocks[:6])
    g2 = nn.Sequential(model.vit_backbone.blocks[6:], model.vit_backbone.norm, model.vit_backbone.head)
    groups = L([g1, g2, model.mlp])
    return groups.map(params)

In [ ]:
# export
# https://keremturgutlu.github.io/self_supervised/
# copy and pasted directly from:
# https://github.com/KeremTurgutlu/self_supervised/blob/main/examples/vision/06%20-%20training_dino_iwang.ipynb
class ViTClassifier(Module):
    def __init__(self, vit_backbone, n_feat_layers, n_classes, lin_f=1024, lin_drop=0.3, pooling='avg'):
        self.vit_backbone  = vit_backbone
        self.n_feat_layers = n_feat_layers 
        self.pooling = pooling
        out_dim = self.vit_backbone.norm.weight.size(0)
        
        if self.n_feat_layers == 1: in_f = 2*out_dim
        else:
            if pooling == 'avg':   in_f = out_dim
            elif pooling == 'cat': in_f = out_dim*n_feat_layers
        
        self.mlp = create_cls_module(in_f, n_classes)
        
    def forward(self,x):
        
        out = self.vit_backbone.get_intermediate_layers(x,self.n_feat_layers)
        
        if self.n_feat_layers == 1:
            # cat [CLS] token and avgpooled output tokens from the last layer
            cls_token, output_tokens = out[0][:,0],out[0][:,1:]
            x = torch.cat([cls_token, output_tokens.mean(1)], dim=1)
        else:
            # avgpool or cat [CLS] tokens from last n layers
            out = [o[:,0] for o in out] 
            if self.pooling == 'avg':   x = torch.stack(out,dim=0).mean(0)
            elif self.pooling == 'cat': x = torch.cat(out, 1)
            else:                       raise Exception("Pooling should be avg or cat")
                
        return self.mlp(x)

In [ ]:
# export
def load_state_dict(arch, pretraining, state_dict_path:str=None, **kwargs):
    '''Get pretrained arch'''
    if arch == "resnet50":
        if pretraining == "sl-imagenet":
            return resnet50(pretrained=True).state_dict()
        elif pretraining == "dino-imagenet":
            # https://github.com/facebookresearch/dino
            return torch.hub.load('facebookresearch/dino:main', 'dino_resnet50').state_dict()
        elif pretraining == "dino-custom":
            dino_state_dict = custom_load(state_dict_path, **kwargs)
            state_dict = OrderedDict()
            for k,v in dino_state_dict.items():
                if "teacher" not in k and k!="C":
                    state_dict[k] = v
            return state_dict
        else:
            raise Exception(f"Unknown pretraining '{arch}'")
            
    elif arch == "deit_small":
        if pretraining == "sl-imagenet":
            # https://github.com/facebookresearch/deit
            return torch.hub.load('facebookresearch/deit:main', 'deit_small_patch16_224', pretrained=True).state_dict()
        elif pretraining == "dino-imagenet":
            # https://github.com/facebookresearch/dino
            return torch.hub.load('facebookresearch/dino:main', 'dino_vits16').state_dict()
        elif pretraining == "dino-custom":
            dino_state_dict = custom_load(state_dict_path, **kwargs)
            state_dict = OrderedDict()
            for k,v in dino_state_dict.items():
                if "teacher" not in k and k!="C":
                    state_dict[k] = v
            return state_dict
        else:
            raise Exception(f"Unknown pretraining '{arch}'")
            
    else:
        raise Exception(f"Unknown architecture '{arch}'")

In [ ]:
# export
def get_dino_arch(arch:str, **kwargs):
    '''We do not care about weights here, simply get the model architecture'''

    if arch == "deit_small": # 21M params
        model = deit_small(**kwargs) # naming artefact (dino fastai library and dino FB library)
    elif arch == "resnet50":
        model = resnet50()
        model.embed_dim = model.fc.weight.shape[1]
        model.fc = Identity()
    else:
        raise Exception(f"Unknown architecture '{arch}'")
        
    return MultiCropWrapper(model)

def get_opt_func(optimizer:str):
    '''get optimizer'''
    if optimizer=="adam":
        return Adam
    elif optimizer=="sgd":
        return SGD
    else:
        raise Exception(f"No optimizer found that matches '{optimizer}'")